In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance, applyGower

In [2]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()
ids = transitionMatrices.index.tolist()
transitionMatrices['id'] = ids

In [3]:
coverList = loadCoverGroups()

In [4]:
df = transitionMatrices
df

,0,1,2,3,4,5,6,7,8,9,...,1286,1287,1288,1290,1291,1292,1293,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,23
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,105
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,118
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,120
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002496
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002497
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002498


In [5]:
ids

[23,
 105,
 118,
 120,
 292,
 407,
 418,
 456,
 467,
 469,
 588,
 628,
 635,
 652,
 816,
 830,
 836,
 1334,
 1345,
 1395,
 1433,
 2063,
 2147,
 2148,
 2220,
 2535,
 2553,
 3118,
 3165,
 3337,
 3391,
 3991,
 4019,
 4041,
 4084,
 4169,
 5643,
 5909,
 6090,
 6097,
 6109,
 6122,
 6124,
 6741,
 8070,
 8272,
 8279,
 8467,
 8619,
 8676,
 8812,
 9297,
 9324,
 9344,
 9362,
 10093,
 10217,
 10226,
 10730,
 10731,
 10735,
 10737,
 10744,
 10759,
 10760,
 10761,
 10796,
 10951,
 11600,
 11603,
 11760,
 12307,
 12376,
 12649,
 12835,
 13305,
 13518,
 13598,
 13936,
 14048,
 14087,
 14172,
 14280,
 14281,
 14296,
 14298,
 14299,
 14300,
 14318,
 14319,
 14322,
 14871,
 14873,
 14874,
 14882,
 14884,
 14943,
 14980,
 15027,
 15165,
 15197,
 15403,
 15421,
 15811,
 15892,
 16066,
 16251,
 16406,
 16491,
 16508,
 16805,
 16882,
 16889,
 17240,
 17250,
 17264,
 17269,
 17273,
 17288,
 17333,
 17335,
 17355,
 17376,
 17384,
 17403,
 17410,
 17413,
 17417,
 17442,
 17443,
 17446,
 17642,
 17773,
 18444,
 

In [6]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
1,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
3,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
4,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5976,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5977,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
5978,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.071429,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.02381,0.0,0.0


In [53]:
pd.DataFrame(coverList)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,180849,6001027,1686718.0,5000013.0,5000001.0,5000007.0,5000008.0,5000023.0,5000017.0,5000019.0,5000020.0,5000018.0,5000011.0,5000012.0
1,6001971,6001984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6002492,6002491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6000699,6001165,6000182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6000104,6000273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6000768,6000784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6001931,6001748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6002043,6002073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6002244,6000194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6001209,6000606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
currentList = coverList[0]
currentList

[180849,
 6001027,
 1686718,
 5000013,
 5000001,
 5000007,
 5000008,
 5000023,
 5000017,
 5000019,
 5000020,
 5000018,
 5000011,
 5000012]

In [68]:
ranksDFGower = applyGower(df,ids, currentList[1],5000)
ranksDFGower

,id,values
0,6001027,0.000000
1,5000011,0.000348
2,5000017,0.000357
3,6000372,0.000642
4,1818565,0.000650
...,...,...
4995,6000912,0.005696
4996,6001031,0.005704
4997,6002409,0.005705
4998,6000568,0.005706


In [69]:
ranksDFGower.loc[ranksDFGower.id.isin(currentList)]

,id,values
0,6001027,0.000000
1,5000011,0.000348
2,5000017,0.000357
13,5000018,0.000927
14,1686718,0.000935
15,5000019,0.000942
31,5000007,0.001094
41,5000013,0.001169
63,5000001,0.001331
241,5000012,0.001763
